In [45]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [46]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [47]:
import nltk
nltk.download('punkt') #for word tokenization
nltk.download('stopwords') #for removing or getting list of stopwords
nltk.download('wordnet') #for lemmatization

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [48]:
from nltk.tokenize import word_tokenize
from nltk.corpus import  stopwords
from nltk.stem import WordNetLemmatizer
import matplotlib.pyplot as plt
from wordcloud import WordCloud

In [49]:
train_data = pd.read_excel("/content/drive/MyDrive/NLP/reviews.train.xlsx" )

In [50]:
train_data

,Sr,Review,Comments
0,0,1,I enrolled for data science course. I got to l...
1,1,1,All the teaching staff and non-teaching staff ...
2,2,1,"IT vedant is very excellent institution, Becau..."
3,3,1,Trusted educational institute. The overall exp...
4,4,1,Itvedant is a good place for you to grow your ...
...,...,...,...
136,136,1,A very good institute where one can make his/h...
137,137,1,This is one of the best institute for the Pyth...
138,138,1,It vedant is best training institute for learn...
139,139,1,Great institute to learn data analysis .Teachi...


In [51]:
train_data.head()

,Sr,Review,Comments
0,0,1,I enrolled for data science course. I got to l...
1,1,1,All the teaching staff and non-teaching staff ...
2,2,1,"IT vedant is very excellent institution, Becau..."
3,3,1,Trusted educational institute. The overall exp...
4,4,1,Itvedant is a good place for you to grow your ...


In [52]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141 entries, 0 to 140
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Sr        141 non-null    int64 
 1   Review    141 non-null    int64 
 2   Comments  141 non-null    object
dtypes: int64(2), object(1)
memory usage: 3.4+ KB


In [53]:
train_data.drop("Sr",axis=1,inplace=True)

In [54]:
train_data

,Review,Comments
0,1,I enrolled for data science course. I got to l...
1,1,All the teaching staff and non-teaching staff ...
2,1,"IT vedant is very excellent institution, Becau..."
3,1,Trusted educational institute. The overall exp...
4,1,Itvedant is a good place for you to grow your ...
...,...,...
136,1,A very good institute where one can make his/h...
137,1,This is one of the best institute for the Pyth...
138,1,It vedant is best training institute for learn...
139,1,Great institute to learn data analysis .Teachi...


In [ ]:
wc = WordCloud(width=800, height=800, background_color="white", min_font_size=10)
wc.generate("".join(train_data[train_data['Review']==1]['Comments']))

plt.figure(figsize=(6,6))
plt.imshow(wc)
plt.axis("off")
plt.show()

In [ ]:
wc = WordCloud(width=800, height=800, background_color="white", min_font_size=10)
wc.generate("".join(train_data[train_data['Review']==0]['Comments']))

plt.figure(figsize=(6,6))
plt.imshow(wc)
plt.axis("off")
plt.show()

In [ ]:
stop = stopwords.words("english")
def clean_text(text):
  tokens = word_tokenize(text.lower())
  # Filter only alphabets
  word_tokens = [t for t in tokens if t.isalpha()]
  clean_tokens = [t for t in word_tokens if t not in stop]
  lemma = WordNetLemmatizer()
  lemma_tokens = [lemma.lemmatize(t) for t in clean_tokens]
  return " ".join(lemma_tokens)

In [ ]:
train_data['Comments'] = train_data['Comments'].apply(clean_text)

In [ ]:
train_data['Comments'].head()

In [ ]:
x_train = train_data['Comments']
y_train = train_data['Review']

In [ ]:
sent_len = []
for t in train_data['Comments']:
  sent_len.append(len(word_tokenize(t)))
train_data['sent_len'] = sent_len
train_data.head()

In [ ]:
max(sent_len)

In [ ]:
np.quantile(sent_len, 0.95)

In [ ]:
max_len=27

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.preprocessing.text import Tokenizer 
# Creates dictionary and every unique word is given number key
from tensorflow.keras.preprocessing import sequence
# To perform the padding of the documents with zero's to make the length of the
# document common
from tensorflow.keras.layers import (LSTM, Dropout, Embedding, SimpleRNN, GRU)
# All the index numbers are converted to vectors using Embedding
# SimpleRNN allows to implement the RNN architecture - activation function -tanh
# Dropout - manage overfitting of model

In [ ]:
# Tokenization
tok = Tokenizer(char_level=False, split=" ")
tok.fit_on_texts(x_train)

In [ ]:
tok.index_word

In [ ]:
vocab_len = len(tok.index_word)
vocab_len

In [ ]:
seq_train = tok.texts_to_sequences(x_train)
seq_train

In [ ]:
seq_padded_train = sequence.pad_sequences(seq_train, maxlen=max_len)
seq_padded_train

In [ ]:
model = Sequential()
# vectorization
model.add(Embedding(vocab_len+1,300, input_length=max_len, mask_zero=True))
# RNN layer
model.add(SimpleRNN(32, activation="tanh"))
# ANN's hidden layer
model.add(Dense(32, activation="relu"))
# To check on overfitting
model.add(Dropout(0.2))
# output layer
model.add(Dense(1, activation="sigmoid"))

In [ ]:
model.summary()

In [ ]:
model.compile(loss="binary_crossentropy", optimizer="adam")

In [ ]:
model.fit(seq_padded_train, y_train, batch_size=50, epochs=20)

In [ ]:
# Loading test data
test_data = pd.read_excel("/content/drive/MyDrive/NLP/reviews.tests.xlsx")
test_data.head()

In [ ]:
test_data.info()

In [ ]:
test_data.shape

In [ ]:
# Apply text cleaning of removal of stop words, puctuation, numbers and lemmitization
test_data['Comments'] = test_data['Comments'].apply(clean_text)

In [ ]:
# Splitting the test data
x_test = test_data['Comments']
y_test = test_data['Review']

In [ ]:
# Converting sentences to token index numbers
seq_test = tok.texts_to_sequences(x_test)
seq_test

In [ ]:
# Adding padding to shorter sentences
seq_padded_test = sequence.pad_sequences(seq_test, maxlen=max_len)
seq_padded_test

In [ ]:
# Predicting the review of the movie using the trained model
y_hat = model.predict(seq_padded_test)

In [ ]:
# y_hat contains probability
y_hat = np.where(y_hat>=0.5, 1, 0)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_hat))

In [ ]:
model = Sequential()
# vectorization
model.add(Embedding(vocab_len+1,300, input_length=max_len, mask_zero=True))
# RNN layer
# model.add(SimpleRNN(32, activation="tanh"))
model.add(LSTM(32, activation="tanh"))
# ANN's hidden layer
model.add(Dense(34, activation="relu"))
# To check on overfitting
model.add(Dropout(0.1))
# output layer
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam")
model.fit(seq_padded_train, y_train, batch_size=100, epochs=50)

In [ ]:
y_hat = model.predict(seq_padded_test)
y_hat = np.where(y_hat>=0.5, 1, 0)
print(classification_report(y_test, y_hat))

In [ ]:
model = Sequential()
# vectorization
model.add(Embedding(vocab_len+1,300, input_length=max_len, mask_zero=True))
# RNN layer
model.add(GRU(34, activation="tanh"))
# ANN's hidden layer
model.add(Dense(38, activation="relu"))
# To check on overfitting|
model.add(Dropout(0.1))
# output layer
model.add(Dense(1, activation="sigmoid"))
model.compile(loss="binary_crossentropy", optimizer="adam")
model.fit(seq_padded_train, y_train, batch_size=150, epochs=50)

Epoch 1/50
1/1 [==============================] - 4s 4s/step - loss: 0.6920
Epoch 2/50
1/1 [==============================] - 0s 66ms/step - loss: 0.6789
Epoch 3/50
1/1 [==============================] - 0s 64ms/step - loss: 0.6661
Epoch 4/50
1/1 [==============================] - 0s 76ms/step - loss: 0.6504
Epoch 5/50
1/1 [==============================] - 0s 66ms/step - loss: 0.6374
Epoch 6/50
1/1 [==============================] - 0s 66ms/step - loss: 0.6211
Epoch 7/50
1/1 [==============================] - 0s 70ms/step - loss: 0.6024
Epoch 8/50
1/1 [==============================] - 0s 69ms/step - loss: 0.5830
Epoch 9/50
1/1 [==============================] - 0s 66ms/step - loss: 0.5625
Epoch 10/50
1/1 [==============================] - 0s 69ms/step - loss: 0.5410
Epoch 11/50
1/1 [==============================] - 0s 62ms/step - loss: 0.5138
Epoch 12/50
1/1 [==============================] - 0s 62ms/step - loss: 0.4844
Epoch 13/50
1/1 [==============================] - 0s 59ms/step

In [ ]:
y_hat = model.predict(seq_padded_test)
y_hat = np.where(y_hat>=0.5, 1, 0)
print(classification_report(y_test, y_hat))

              precision    recall  f1-score   support

           0       0.33      0.33      0.33         3
           1       0.96      0.96      0.96        54

    accuracy                           0.93        57
   macro avg       0.65      0.65      0.65        57
weighted avg       0.93      0.93      0.93        57

